# Zonal energy spectrum - ERA5

* This notebook shows how zonal energy spectrum was computed for kinetic energy and potential temperature energy. For the latter, the 500 hPa potential temperature was approximated using $\theta=T(1000/500)^{(287.0/1004)}$.

In [1]:
import os
import sys
import yaml
from glob import glob
from datetime import datetime, timedelta

import numpy as np
import xarray as xr

sys.path.insert(0, os.path.realpath('../libs/'))
import verif_utils as vu
import score_utils as su

In [11]:
from weatherbench2.derived_variables import ZonalEnergySpectrum

### Load config

In [23]:
config_name = os.path.realpath('verif_config.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

### ZES compute

In [3]:
variables_levels = {
    'T500': None,
    'U500': None,
    'V500': None
}

In [24]:
path_verif = conf['ERA5_ours']['save_loc_clim']+'combined_zes_clim.nc'

In [7]:
# ---------------------------------------------------------------------------------------- #
# ERA5
filename_ERA5 = sorted(glob(conf['ERA5_ours']['save_loc']))

# pick years (use IFS years)
year_range = conf['IFS']['year_range']
years_pick = np.arange(year_range[0], year_range[1]+1, 1).astype(str)
filename_ERA5 = [fn for fn in filename_ERA5 if any(year in fn for year in years_pick)]

# merge yearly ERA5 as one
ds_ERA5 = [vu.get_forward_data(fn) for fn in filename_ERA5]
ds_ERA5_merge = xr.concat(ds_ERA5, dim='time')

# subset merged ERA5 and unify coord names
ds_ERA5_merge = vu.ds_subset_everything(ds_ERA5_merge, variables_levels)

L_time = len(ds_ERA5_merge['time'])

In [12]:
verif_results = []

for i_time in range(L_time):
    ds_ERA5 = ds_ERA5_merge.isel(time=i_time)    
    ds_ERA5 = ds_ERA5.load()

    # -------------------------------------------------------------- #
    # potential temperature
    ds_ERA5['theta500'] = ds_ERA5['T500'] * (1000/500)**(287.0/1004)
    # -------------------------------------------------------------- #
    zes_temp = []
    for var in ['U500', 'V500', 'theta500']:
        calc = ZonalEnergySpectrum(var)
        result_fft = calc.compute(ds_ERA5)
        zes_temp.append(result_fft.rename(var))
        
    verif_results.append(xr.merge(zes_temp))

ds_verif = xr.concat(verif_results, dim='time')

In [13]:
ds_verif

<xarray.Dataset>
Dimensions:           (latitude: 192, time: 4384, zonal_wavenumber: 145)
Coordinates:
  * latitude          (latitude) float32 90.0 89.06 88.12 ... -89.06 -90.0
  * time              (time) datetime64[ns] 2020-01-01 ... 2022-12-31T18:00:00
  * zonal_wavenumber  (zonal_wavenumber) int64 0 1 2 3 4 ... 140 141 142 143 144
    frequency         (zonal_wavenumber, latitude) float64 -0.0 0.0 ... -82.34
    wavelength        (zonal_wavenumber, latitude) float64 -inf inf ... -0.01214
Data variables:
    U500              (time, latitude, zonal_wavenumber) float64 -0.5134 ... ...
    V500              (time, latitude, zonal_wavenumber) float64 -0.04241 ......
    theta500          (time, latitude, zonal_wavenumber) float64 -1.401e+05 ....

In [25]:
path_verif

'/glade/campaign/cisl/aiml/ksha/CREDIT_CESM/VERIF/ERA5_clim/combined_zes_clim.nc'

In [26]:
# ds_verif.to_netcdf(path_verif)